# Create database

1. Upload the compressed file
1. Unzip the file
1. Assign how many items to search from openfood
1. Assign how many products to look into from the search results page

In [1]:
# from google.colab import files
# files.upload()

In [2]:
# !unzip retailer_analysis

In [3]:
retailer = "Tesco"
csvfile = "openfood_tesco.csv"

# 0~5251, cuz that's the rows number in openfood tesco
from_num = 2001
to_num = 2500
search_items_per_requests=4

In [4]:
import pandas as pd

# only select the product names
open_products_df = pd.read_csv(csvfile, sep="\t", dtype=str).dropna(subset=['product_name_en'])['product_name_en']
open_products_list = open_products_df.astype(str).values.tolist()

open_products_list = open_products_list[from_num:to_num]

In [5]:
from item.Item import *
from crawler.TescoCrawler import TescoCrawler

crawler = TescoCrawler()
completed_info_products = []
for each in open_products_list:
    search_item = Search_Item(each)
    search_item = crawler.search_and_append(search_item, num=search_items_per_requests)
    completed_info_products.append(search_item.tesco_items)

Start searching item: "Tesco Unsmoked Bacon" at Tesco
looking into https://www.tesco.ie/groceries/en-IE/products/280303323
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/302430949
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/292621354
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/303239387
response: 200
Start searching item: "Omega 3 Fish Oil" at Tesco
looking into https://www.tesco.ie/groceries/en-IE/products/309756992
response: 200
Start searching item: "Prosecco Rose Brut Millesimato" at Tesco
looking into https://www.tesco.ie/groceries/en-IE/products/295037230
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/309783043
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/306802729
response: 200
looking into https://www.tesco.ie/groceries/en-IE/products/310155587
response: 200
Start searching item: "chicken leg" at Tesco
looking into https://www.tesco.ie/grocer

# Prepare the data
1. Flatten the nested products list
2. create the dataframe from the item object
    1. retrieve the object variables
    2. create a dataframe
    3. add the dataframe to the frames list
    4. concatenate to create the entire dataframe
3. Save to database with name `{retainer}_{from_source_item_num}_{to_source_item_num}`

In [6]:
completed_info_products = [i for sublist in completed_info_products for i in sublist]
print(f"Totally crawls {len(completed_info_products)} products")

Totally crawls 1604 products


# Save the result to db

In [7]:
import sqlite3

frames = []
for e in completed_info_products:
    product_info_dict = vars(e)
    product_info_dict['nutrition'] = str(product_info_dict['nutrition'])
    product_df = pd.DataFrame(product_info_dict, index=[0], dtype=str)
    frames.append(product_df)

result = pd.concat(frames)

database_name = f"{retailer}_{from_num}_{to_num}.db"
conn = sqlite3.connect(database_name)
result.to_sql("products", conn, if_exists="replace", index=False)
f"Save to {database_name}"

'Save to Tesco_2001_2500.db'